In [1]:
#!pip install pdfminer3

In [2]:
from pdfminer3.layout import LAParams, LTTextBox
from pdfminer3.pdfpage import PDFPage
from pdfminer3.pdfinterp import PDFResourceManager
from pdfminer3.pdfinterp import PDFPageInterpreter
from pdfminer3.converter import PDFPageAggregator
from pdfminer3.converter import TextConverter
from pymongo import MongoClient
import io
import re
import os

In [3]:
class SIC:
    def __init__(self):
        self.mongodb_collection = None

    def eliminar_pie_de_pagina(self, text):
        pattern = r'\n+[A-Z]+[0-9]+-[A-Z]+[0-9]+ \([0-9]{4}-[0-9]{2}-[0-9]{2}\) *\n+( *REPÚBLICA DE COLOMBIA *)?'
        return re.sub(pattern, '\n', text)

    def eliminar_encabezado(self, text):
        pattern = r'[\n\x0c ]*(SENTENCIA|Sentencia) *(N[UÚ]MERO|No|N°)[0-9A-GI-Za-gi-z\n\_\.° ]+((HOJA|Hoja|hoja)[No°\. ]+[0-9\/\_ ]+)*\n\n'
        return re.sub(pattern, '\n', text)
    
    def eliminar_numeracion(self, text):
        pattern = r' *Página [0-9]+ de [0-9]+ *'
        return re.sub(pattern, '\n', text)
    
    def eliminar_saltos_de_linea(self, text):
        pattern = r'((?<=[a-záéíóúü0-9°º\(\),;\-\"\']) ?\n+ ?(?=[a-záéíóúü0-9°º\(\)\,\-\"\'\:]+(\n| ))|(?<=[0-9I]\.)(\n+)(?=[A-Za-z]))'
        return re.sub(pattern, ' ', text)

    def obtener_numero_sentecia(self, text):
        pattern =r'((?<=(SENTENCIA NÚMERO))[ 0-9]+\n|(?<=((SENTENCIA|Sentencia) No.))[0-9 ]+\n)'
        try:
            return re.search(pattern, text)[0]
        except:
            return re.search(pattern, text)
        
    def obtener_numero_expediente(self, text):
        pattern =r'(?<=(Expediente:))[ 0-9]+\n'
        try:
            return re.search(pattern, text)[0]
        except:
            return re.search(pattern, text)
        
    def obtener_numero_radicado(self, text):
        pattern =r'(?<=(Radicado No.))( *)?[0-9\- ]'
        try:
            return re.search(pattern, text)[0]
        except:
            return re.search(pattern, text)

    def cortar_secciones(self, text, indices):
        secciones = [(0, indices[0])]
        for i in range(len(indices)-1):
            secciones.append((indices[i],indices[i+1]))
        secciones.append((indices[-1],len(text)))
        return secciones

    def obtener_secciones(self, text):
        patterns = [r'ANTECEDENTES([ \:\.]*)?\n',
                    r'(CONSIDERACIONES|CONSIDERACIONES DEL DESPACHO)([ \:\.]*)?\n', 
                    r'(DECISI[OÓ]N([ \:\.\,]*)?\n|RESUELVE([ \:\.\,]*)?\n)',
                    r'NOTIF[IÍ]QUESE([ \:\.\,]*)?\n']
        indices = []
        for pattern in patterns:
            indice = re.search(pattern, text)
            if indice:
                indices.append(indice.span()[0])
        
        return self.cortar_secciones(text, indices)
    
    def conectar_mongo(self, database, collection, connection_local=True):
        '''
        Hace conexión con la base de datos de MongoDB.
        
        - database: De formato String. Es el nombre de la base de datos a conectarse. Si la base de datos ya existe se conecta a esta, si no existe la crea y se conecta.
        - collection: De formato String. Es el nombre de la colección a conectarse. Si la colección ya existe se conecta a esta, si no existe la crea y se conecta.
        - connection_local: Si es "True" se conecta de manera local, si es "False" hace conexión remota a Atlas. Es importante tener el archivo "URI.txt" con el uri para ralizar la conexión remota.
        '''
        
        # Inicia la conexión
        try:
            if connection_local:
                cliente = MongoClient()
            else:
                # Traemos la clave
                with open('URI.txt', 'r') as f:
                    uri = f.readline()
                cliente = MongoClient(uri)
            
            # Probamos que la conexión haya sido exitosa                
            cliente.admin.command('ismaster')
            print("Conexión exitosa")
            
            # Ingresa los datos de la base de datos
            db = cliente[database]
            col = db[collection]
            self.mongodb_collection = col
            print('Se encuentra conectado a MongoDB', 
                  f'Base de datos: {database}', 
                  f'Colección: {collection}',
                  sep='\n')
            return True
            
        except:
            print("No se pudo conectar al servidor")
            return False

    def procesamiento(self, path):
        resource_manager = PDFResourceManager()
        fake_file_handle = io.StringIO()
        converter = TextConverter(resource_manager, fake_file_handle, laparams=LAParams())
        page_interpreter = PDFPageInterpreter(resource_manager, converter)

        with open(path, 'rb') as fh:
        
            for page in PDFPage.get_pages(fh, caching=True, check_extractable=True):
                page_interpreter.process_page(page)

            text = fake_file_handle.getvalue()

        converter.close()
        fake_file_handle.close()

        num_sentencia = self.obtener_numero_sentecia(text)
        num_radicado = self.obtener_numero_radicado(text)
        num_expediente = self.obtener_numero_expediente(text)
        
        text = self.eliminar_pie_de_pagina(text)
        text = self.eliminar_encabezado(text)
        text = self.eliminar_numeracion(text)
        text = self.eliminar_saltos_de_linea(text)
        
        secciones = self.obtener_secciones(text)
        
        sentencia = {}
        
        if num_sentencia:
            sentencia = sentencia | {'numero_sentencia': num_sentencia}
        if num_radicado:
            sentencia = sentencia | {'numero_radicado': num_sentencia}
        if num_expediente:
            sentencia = sentencia | {'numero_expediente': num_expediente}
        if text:
            sentencia = sentencia | {'sentencia': text}
        if secciones:
            sentencia = sentencia | {'secciones': secciones}
        
        if sentencia:
            self.mongodb_collection.insert_one(sentencia)
        else:
            print('No se guardó')


In [4]:
sic = SIC()

database = 'sic'
collection = 'sentencias'
sic.conectar_mongo(database=database, collection=collection)

directorio = 'Sentencias'

for file in os.listdir(directorio):
    path = os.path.join(directorio, file)
    print(path)
    sic.procesamiento(path=path)


Conexión exitosa
Se encuentra conectado a MongoDB
Base de datos: sic
Colección: sentencias
Sentencias/Sentencia_06_2010.pdf
[739, 4992, 30894, 32140]
Sentencias/Sentencia 8405 de 29 jul 2021.pdf
[755, 3952, 28074, 35934]
Sentencias/Sentencia_05_2010.pdf
[915, 15204, 53867, 54662]
Sentencias/Sentencia_04_2010.pdf
[572, 6788, 23372, 23888]
Sentencias/Sentencia_03_2010.pdf
[606, 5441, 15344, 15755]
Sentencias/Sentencia_15_2010.pdf
[698, 8731, 39991, 40485]
Sentencias/Sentencia_01_2010.pdf
[614, 9601, 112457, 113371]
Sentencias/Sentencia_14_2010.pdf
[553, 10967, 59181, 60676]
Sentencias/sentencia_11966.pdf
[643, 3845, 18314, 21191]
Sentencias/Sentencia 21-32299.pdf
[743, 4130, 23531, 28750]
Sentencias/Sentencia_09_2010.pdf
[536, 6422, 44749, 46362]
Sentencias/Sentencia_13_2009.pdf
[704, 5652, 28781, 28945]
Sentencias/Sentencia_12_2010.pdf
[617, 4365, 29420, 30204]
Sentencias/Sentencia_08_2010.pdf
[578, 7140, 36682, 37036]
Sentencias/Sentencia_02_2010.pdf
[648, 7365, 34972, 35419]
Sentencia